In [1]:
import pandas as pd
from google.colab import drive


drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
data_APT = pd.read_csv('/content/drive/MyDrive/Ultimate_Data/data_APT.csv', encoding='utf-8')
data_APT.info()
data_APT

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93453 entries, 0 to 93452
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Building_Age                     93453 non-null  int64  
 1   JS_Price                         93453 non-null  int64  
 2   JS_BA                            93453 non-null  float64
 3   Population                       93453 non-null  int64  
 4   LC_index                         93453 non-null  float64
 5   TC_index                         93453 non-null  float64
 6   SDT_index                        93453 non-null  float64
 7   Sell_Price                       93453 non-null  float64
 8   Crime_Rates                      93453 non-null  float64
 9   IR                               93453 non-null  float64
 10  Region_강남구                       93453 non-null  int64  
 11  Region_강동구                       93453 non-null  int64  
 12  Region_강북구        

,Building_Age,JS_Price,JS_BA,Population,LC_index,TC_index,SDT_index,Sell_Price,Crime_Rates,IR,...,Region_용산구,Region_은평구,Region_종로구,Region_중구,Region_중랑구,Year,Month,Shortest_Distance_to_Subway,Shortest_Distance_to_University,Shortest_Distance_to_School
0,15,18000,59.97,299533,72.000000,74.200000,114.099327,31400.00,0.930480,2.75,...,0,0,0,0,0,2011,1,744.388888,1783.606149,370.003165
1,32,30000,84.43,557563,72.000000,74.200000,113.009872,100404.17,0.842083,2.75,...,0,0,0,0,0,2011,1,839.698707,3723.160551,336.123079
2,14,22000,59.82,368260,72.000000,74.200000,113.009872,56836.67,0.836749,2.75,...,0,0,0,0,0,2011,1,351.857009,1517.638732,470.686016
3,25,18000,47.94,494724,72.000000,74.200000,114.099106,82850.00,0.943467,2.75,...,0,0,0,0,0,2011,1,907.582994,3053.953581,180.644334
4,12,23500,149.97,420803,72.000000,74.200000,114.270256,34030.00,0.854310,2.75,...,0,0,0,0,0,2011,1,248.141179,718.700972,318.836190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93448,26,45000,84.20,384272,102.715637,100.768990,132.138962,54000.00,0.779744,3.50,...,0,0,0,0,1,2023,9,255.627811,1952.815142,188.219471
93449,13,40000,59.60,291384,102.715637,100.768990,132.138962,107500.00,0.779744,3.50,...,0,0,0,0,0,2023,9,891.504266,658.072434,238.994104
93450,24,45000,59.97,121441,102.715637,100.768990,132.138962,125000.00,0.779744,3.50,...,0,0,0,1,0,2023,9,256.095668,1840.603558,167.573972
93451,31,30000,58.65,566839,102.715637,100.768990,132.138962,53000.00,0.779744,3.50,...,0,0,0,0,0,2023,9,494.719586,1701.812476,171.601187


In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset (assuming you have already loaded it into a DataFrame called 'df')
# Replace 'df' with your actual DataFrame name
df = data_APT

# Split the data into features (X) and target variable (y)
X = df.drop("Sell_Price", axis=1)
y = df["Sell_Price"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=283)

# Set hyperparameters for XGBoost model (adjust these based on your requirements)
params = {
    'objective': 'reg:squarederror',
    'booster': 'gbtree',
    # Add other hyperparameters as per your requirement
}

# Perform RandomizedSearchCV for hyperparameter tuning
model_xgb = xgb.XGBRegressor()
random_grid = {
    'n_estimators': (300, 1500),
    'learning_rate': (0.01, 0.1),
    'max_depth': (3, 10),
    'min_child_weight': (1, 10),
    'gamma': (0, 1),
    'subsample': (0.6, 1),
    'colsample_bytree': (0.6, 1),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1)
}

rscv = RandomizedSearchCV(estimator=model_xgb,
                          param_distributions=random_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          n_iter=100,
                          n_jobs=-1,
                          verbose=1,
                          random_state=742)

rscv.fit(X_train, y_train)

print("Best parameters found: ", rscv.best_params_)

# Train the model using XGBoost with best parameters
model = xgb.XGBRegressor(**params, **rscv.best_params_)
model.fit(X_train, y_train)

# Make predictions on the testing set using trained model
y_pred = model.predict(X_test)

# Calculate evaluation metrics: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error),
# MAPE (Mean Absolute Percentage Error), R-Squared

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("R-Squared:", r2)

# Plot actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='b', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Sell_Price')
plt.ylabel('Predicted Sell_Price')
plt.title('Actual vs Predicted Sell_Price')
plt.show()

# Create a histogram of residuals with MAE and MAPE
residuals = y_test - y_pred
sns.histplot(residuals, kde=True)
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Histogram of Residuals\nMAPE: {:.3f}, R-squared: {:.3f}".format(mape, r2))
plt.grid(True)